In [1]:
import os
os.environ [ "KMP_DUPLICATE_LIB_OK" ]= "TRUE"

%load_ext autoreload
%matplotlib notebook
%autoreload 2

In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/12 09:54
# @Author  : Wang Yujia
# @File    : SA_for_PT_one_for_all.ipynb
# @Description : for all settings and data, just infer one params

# 0. What for
1. 给所有的data和setting，只infer一组params

# 1. Preparations
## 1.1 全局设置

In [3]:
# 人工数据集or真实数据集
ARTIFICIAL = True
# seed = [3,31,204,223,407,62,508,626]
# [4,31,204,35,407,66,508,512]

seed = 512
noise_pct = 0.05

SET_VAL = False
train_pct = 0.7
vali_pct = 0.2
test_pct = 0.1


# Small dataset
data_small_np_path = r'../../data/small_auctions_np.csv'
settings_small_NN_path = r"../../data/small_settings_NN.csv"

# Large data
data_large_np_path = r'E:\DATA\large_dta\large_auctions_np.csv'
settings_large_NN_path = r'E:\DATA\large_dta\large_settings_NN.csv'

# data
if ARTIFICIAL:
    target_root_path = "../../data/artificial_targets_v2_noise=" +str(noise_pct)
else:
    target_root_path = "../../data/targets_all"

# params_opitim_path = r"../../data/params_and_K_sampled.csv"

unique_setting_NN = ['desc','bidincrement','bidfee','retail','flg_endprice']
LEN = 300

# output path
if ARTIFICIAL:
    params_opitim_unified_path = r"../../data/SA_PT/params_artificial_unified_noise="+str(noise_pct)+ "_seed="+str(seed)+".csv"
else:
    params_opitim_unified_path = "../../data/SA_PT/params_opitim_unified_seed="+str(seed)+".csv"

import numpy as np
import csv
import os
import pandas as pd
from visdom import Visdom
from SA_for_PT_funcs_delta_eq1 import *
from SA_modified import SABoltzmann
from sko.tools import set_run_mode
from func_timeout import func_set_timeout
import func_timeout
from tqdm import tqdm
import json
from MLP.utils import setup_seed


In [4]:
# for SA
# initial params
table_5_M = [0.025,3.72]
# lower/ upper bound
lb = [-0.3,0.01]
ub = [0.3,18]
# viz = Visdom(env='P',use_incoming_socket=False)
max_trial = 3       # 算法执行次数，会产生max_trial个估计值


## 1.2 data 读取
1. 注意把small和large拼接成一张data table
2. 由于columns不同，合并后data中肯定有nan，不过我们需要的columns不是nan就好

In [5]:
data_small = pd.read_csv(data_small_np_path, encoding="utf-8")
data_large = pd.read_csv(data_large_np_path, encoding="utf-8")
data = pd.concat([data_small,data_large],axis=0,ignore_index=True)
# 由于columns不同，合并后data中肯定有nan，不过我们需要的columns不是nan就好

data_key_small = pd.read_csv(settings_small_NN_path, encoding="utf-8")
data_key_large = pd.read_csv(settings_large_NN_path, encoding="utf-8")
data_key = pd.concat([data_key_small,data_key_large],axis=0,ignore_index=True)

# 有`N_uniq_auction`组setting
N_uniq_auction= data_key.shape[0]
print("For PT model, there are *{}* settings waiting to be inferred.".format(N_uniq_auction))
data_key.head()

For PT model, there are *1276* settings waiting to be inferred.


,desc,bidincrement,bidfee,retail,flg_endprice
0,Sony Ericsson S500i Unlocked Mysterious Green,0.15,0.75,499.99,0
1,PSP Slim & Lite Sony Piano Black,0.15,0.75,169.99,0
2,iPod Touch Apple 8GB with Software Upgrade,0.15,0.75,299.99,0
3,Logitech Cordless Wave Keyboard and Mouse,0.15,0.75,89.99,0
4,SanDisk Cruzer Contour 4GB,0.15,0.75,59.99,0


## 1.3 functions about 'key'

In [6]:
all_target_file = os.listdir(target_root_path)

In [7]:
def select_data_fromkey(i):
    return data[(data['desc'] == data_key.iloc[i,0]) &
                (data['bidincrement'] == data_key.iloc[i,1]) &
                (data['bidfee'] == data_key.iloc[i,2]) &
                (data['retail'] == data_key.iloc[i,3]) &
                (data['flg_endprice'] == data_key.iloc[i,4])].copy()

def readin_data_fromkey(i):
    data_i_path = os.path.join(target_root_path,all_target_file[i])
    data_i = pd.read_csv(data_i_path,encoding="utf-8")
    return data_i

## 1.4 seed


In [10]:
setup_seed(seed)

DATA_len = len(target_root_path)

shuffled_indices = np.random.permutation(DATA_len)

train_idx = shuffled_indices[:int(train_pct * DATA_len)]
if SET_VAL:
    tmp = int((train_pct + vali_pct) * DATA_len)
    val_idx = shuffled_indices[int(train_pct * DATA_len):tmp]
    test_idx = shuffled_indices[tmp:]
else :
    tmp = int((train_pct + vali_pct) * DATA_len)
    test_idx = shuffled_indices[int(train_pct * DATA_len):tmp]
    val_idx = shuffled_indices[tmp:]

In [11]:
len(test_idx)

8

# 2. SA
## 2.1 define loss function
1. loss function: NLL for auctions with same `features_GT`
2. “one for all”的不同之处在于，累加所有setting的情况求一个loss，不再区分`data_i`

In [12]:
def loss_func(params,other_params):
    # start_time_lo\ss = datetime.datetime.now()
    alpha = params[0]
    # delta = 1
    labda = params[1]

    # Sum up nll for all configs
    NLL=0.0

    # For those in training set
    for i in train_idx:

        data_i = readin_data_fromkey(i)

        # # Extract data with same `key_i` into a table
        # data_i = select_data_fromkey(i)
        # data_i.reset_index(drop=True,inplace=True)

        # Get params
        v = float(data_key.iloc[i,3])
        d = float(data_key.iloc[i,1])
        b = float(data_key.iloc[i,2])
        T_i = data_i['N'].astype(int)            # auction duration sequence

        LEN,T = get_LEN_T(v,b,d,max(T_i))

        # To get U_i given LEN and other params
        # 如果eps设置为0，则可能出现loss=inf的情况（log(1-1)导致的）
        U_i = get_U_GT2(LEN, v, d, b, alpha, labda)

        # Calculate NLL loss under this auction setting & PT params
        nll = get_nll_loss(T_i, U_i, LEN)

        # Sum up nll for all configs
        NLL += nll
    print(f"NLL loss is {NLL}")
    return float(NLL)

## 2.2 do SA
1. 要对所有settings做一次infer
2. 具体的：对每个setting `i`
    - 每一个setting `i` 可以提取出来一个`data_i`，代表所有auction
    - 每一个`data_i`中的`cnt_uniq`，也就是`A`，是相同的，表示setting `i` 进行的拍卖总次数.【但是这个`A`在计算loss的时候派不上用场】
    - `N`表示duration，因此paper公式里的$T_a$即`N[a]`
    - 因此有`A = sum(data_i['cnt_n_2'])`，其中的'cnt_n_2'表示了该行对应的`duration=N`发生的次数
    - 按照上文，求解`U[i]_t` which is a array with shape of (max(N)),也就是求解paper里的`p_t`
3.每次进行`L`次对参数的试探寻找，每次寻找对应一个温度一组新的参数。
    - 优化的完成/退出条件：温度小于`T_min`或者最低温度保持`max_stay_counter`次的不变
    - 鉴于温度小于`T_min`很难达到，因此基本上对一组参数进行优化要进行L*max_stay_counter+1次运算（loss运算）

In [13]:
params_opitim = pd.DataFrame(columns=['trial_time','alpha','delta','labda','initial_loss','final_loss','avg_loss'])

In [14]:
# Perform SA for all settings at once
for i in range(0,max_trial):
    print(f"> For the *{i}_th* trial")
    # print("> Initilizing SA....... \n")
    set_run_mode(loss_func, 'cached')
    set_run_mode(loss_func, 'multithreading')

    initial_t = max(data['cnt_N'].astype(int))        # initial temperature

    # other_params = [max_T,v,d,b,T_i,cnt_n_2_i]
    sa_boltzmann = SABoltzmann(func=loss_func, x0=table_5_M, other_params = [], T_max=initial_t, T_min=1, learn_rate=0.2, L=20, max_stay_counter=10, lb=lb, ub=ub)

    print("> Now do SA....... \n")
    best_x, best_y = sa_boltzmann.run()
    # print('> The whole inference process costs {time_costs}s \n'.format(time_costs=(datetime.datetime.now() - start_time).total_seconds()))

    print("> SA ENDS....... \n")

    # Append the opitimized params of this trial
    df_tmp = pd.DataFrame([ [i,best_x[0],1,best_x[1],sa_boltzmann.generation_best_Y[0],best_y,best_y/data.shape[0]] ], columns=['trial_time','alpha','delta','labda','initial_loss','final_loss','avg_loss'])
    params_opitim = params_opitim.append(df_tmp,ignore_index=True)

> For the *0_th* trial
NLL loss is 62854.666695331136
> Now do SA....... 

NLL loss is 110128.01567383278
NLL loss is 63249.08394861729
NLL loss is 66814.83657259932
NLL loss is 74629.43330652083
NLL loss is 67159.43122089804
NLL loss is 109273.65117551466
NLL loss is 125080.75003436537
NLL loss is 79495.98522086976
NLL loss is 65954.17250542785
NLL loss is 70373.90898482403
NLL loss is 63231.593897289735
NLL loss is 67727.37751678341
NLL loss is 115659.09270959283
NLL loss is 115901.13395920114
NLL loss is 110080.2426829309
NLL loss is 108351.39333268412
NLL loss is 108167.02115382552
NLL loss is 61362.43912956679
NLL loss is 122227.3045313829
NLL loss is 116794.58987649101
NLL loss is 61751.75480942213
NLL loss is 73655.85948869018
NLL loss is 85067.22868533588
NLL loss is 60620.70427117836
NLL loss is 145818.2735351034
NLL loss is 83222.9773344272
NLL loss is 133128.28865542548
NLL loss is 60651.23199784014
NLL loss is 80293.62775502454
NLL loss is 75152.89117657118
NLL loss is 6490

C:\Users\Administrator\AppData\Local\Temp\ipykernel_22204\1801197125.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  params_opitim = params_opitim.append(df_tmp,ignore_index=True)


NLL loss is 62854.666695331136
> Now do SA....... 

NLL loss is 117980.37580788058
NLL loss is 112452.55653339883
NLL loss is 106541.8803242039
NLL loss is 183080.10785199716
NLL loss is 74189.46583915813
NLL loss is 74247.76454306192
NLL loss is 98494.47799502859
NLL loss is 64066.064282314794
NLL loss is 93010.33402046132
NLL loss is 62993.68501204375
NLL loss is 147547.92890211314
NLL loss is 180769.89762203622
NLL loss is 168495.48610491003
NLL loss is 173526.36361822858
NLL loss is 99707.29655470043
NLL loss is 110799.43047959016
NLL loss is 62141.993783282414
NLL loss is 88604.43054219754
NLL loss is 70721.66175808814
NLL loss is 75966.94252203927
NLL loss is 79157.02425851396
NLL loss is 94912.25875834165
NLL loss is 153745.4589789418
NLL loss is 69988.10870683819
NLL loss is 69552.96638783213
NLL loss is 115553.48769404515
NLL loss is 77101.52384969109
NLL loss is 78725.15546165443
NLL loss is 67624.29206915708
NLL loss is 75405.15052438718
NLL loss is 63479.73658678973
NLL los

C:\Users\Administrator\AppData\Local\Temp\ipykernel_22204\1801197125.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  params_opitim = params_opitim.append(df_tmp,ignore_index=True)


NLL loss is 62854.666695331136
> Now do SA....... 

NLL loss is 66822.23953867434
NLL loss is 61690.479721584816
NLL loss is 110240.37128206395
NLL loss is 133205.0200055679
NLL loss is 112945.86517125396
NLL loss is 113215.50837375683
NLL loss is 115883.1808147543
NLL loss is 79653.27952567131
NLL loss is 179776.50232385617
NLL loss is 61900.6535587946
NLL loss is 108575.74978036332
NLL loss is 66067.94357401844
NLL loss is 89988.07507624106
NLL loss is 132251.6138663396
NLL loss is 68083.45756843538
NLL loss is 108605.27090320156
NLL loss is 116875.3508181698
NLL loss is 139667.86045887275
NLL loss is 62391.60423787479
NLL loss is 208922.63959546777
NLL loss is 133457.40774151718
NLL loss is 99932.12400023683
NLL loss is 115341.22224902383
NLL loss is 106738.49918106456
NLL loss is 63884.65107015313
NLL loss is 63390.92245272638
NLL loss is 125488.67486654491
NLL loss is 108212.31270791173
NLL loss is 84682.28773054246
NLL loss is 71170.93915944462
NLL loss is 61816.77424004545
NLL l

C:\Users\Administrator\AppData\Local\Temp\ipykernel_22204\1801197125.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  params_opitim = params_opitim.append(df_tmp,ignore_index=True)


In [15]:
# save 'params_opitim' for later check
params_opitim.to_csv(params_opitim_unified_path, header=True, encoding="utf-8",index=False)

In [16]:
params_opitim_unified_path

'../../data/SA_PT/params_artificial_unified_noise=0.05_seed=512.csv'